In [39]:
import madina as md
import geopandas as gpd
import pandas as pd
import math
from shapely.geometry import box
import madina.una.tools as una
import numpy as np

In [40]:
nycstreet = gpd.read_file("../01_data/03_intermediate/cut/cut.shp")
index_with = gpd.read_file("../01_data/03_intermediate/index/index.shp")
nycstreet = nycstreet.to_crs('EPSG:32619')
index_with = index_with.to_crs('EPSG:32619')

In [41]:
# degree
# Load Layers
nyc = md.Zonal()
nyc.load_layer('street', source=nycstreet)
nyc.load_layer('collison', source=index_with)

# Create network
nyc.create_street_network(source_layer="street")
nyc.insert_node(label='origin', layer_name="collison")

In [42]:

relaion = nyc.network.nodes[nyc.network.nodes['type'].isin(['origin'])]

In [43]:
crashnode = nyc.network.nodes[nyc.network.nodes['type'].isin(['origin'])].reset_index()
crashnode = crashnode[['id','source_id','nearest_edge_id','edge_start_node','edge_end_node']]
streetnode = nyc.network.nodes[nyc.network.nodes['type'].isin(['street_node'])].reset_index()[['id','degree']]
streetnode['edge_start_node'] = streetnode['id'] 
streetnode = streetnode.drop('id',axis=1)

In [44]:
crashnode = pd.merge(crashnode,streetnode,on = 'edge_start_node',how='left' )

In [45]:
streetnode['edge_end_node'] = streetnode['edge_start_node'] 
streetnode = streetnode.drop('edge_start_node',axis=1)
crashnode = pd.merge(crashnode,streetnode,on = 'edge_end_node',how='left' )

In [46]:
crashnode['degree'] = crashnode['degree_x']+crashnode['degree_y']-2

In [47]:
index1 = index_with.drop(['id', 'geometry', 'ON STREET', 'CROSS STRE', 'OFF STREET'],axis=1)
index1['death'] = index1['NUMBER O_1'] + index1['NUMBER O_3'] + index1['NUMBER O_5'] + index1['NUMBER O_7']
index1['injury'] = index1['NUMBER OF'] + index1['NUMBER O_2'] + index1['NUMBER O_3'] + index1['NUMBER O_6']
index1['property'] = index1[['VEHICLE _1', 'VEHICLE _2', 'VEHICLE _3', 'VEHICLE _4']].isna().sum(axis=1)

In [48]:
index1['severity'] = index1['death'] + index1['injury']*0.6 + index1['property']*0.4

In [49]:
index1['severity'].unique()

array([ 1.6,  1.2,  1.8,  2.4,  2.8,  3.6,  4. ,  5.6,  0.8,  2. ,  6. ,
        2.2,  0.4,  4.8,  3.2,  4.4,  6.4,  5. ,  2.4,  1.2,  7.2,  0. ,
        5.2,  5.2,  4. ,  8.4,  1.8,  2.6,  3.4,  1.4,  6.8,  4.2,  9.6,
        1. ,  3. ,  4.4,  2.6,  7.6,  3.6,  4.6,  8. ,  7.2,  5.6,  8. ,
        9.2,  5.4, 16.4,  8.8,  7.4, 14.4, 18. , 10.4,  8.4,  6.6, 19.2,
        6.6, 11.2,  4.8, 10.8,  0.6,  8.8, 20.4, 11.6,  3.8, 14. , 16.8,
        3.4,  2.2,  6.2, 22.8,  6.4, 12.8, 12. ,  3.8, 10. , 21.6, 13.2,
        8.2, 11. ,  8.6, 14.2, 28.8, 12.4, 42. , 17.2, 13.4, 15.2,  5.8,
       15.6,  9.6, 10.8, 18.4])

In [50]:
result = pd.merge(index1,crashnode,left_on = index1.index, right_on = "source_id", how = 'left')

In [51]:
result.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'NUMBER OF', 'NUMBER O_1', 'NUMBER O_2',
       'NUMBER O_3', 'NUMBER O_4', 'NUMBER O_5', 'NUMBER O_6', 'NUMBER O_7',
       'CONTRIBUTI', 'CONTRIBU_1', 'CONTRIBU_2', 'CONTRIBU_3', 'CONTRIBU_4',
       'COLLISION_', 'VEHICLE TY', 'VEHICLE _1', 'VEHICLE _2', 'VEHICLE _3',
       'VEHICLE _4', 'gravity_to', 'death', 'injury', 'property', 'severity',
       'id', 'source_id', 'nearest_edge_id', 'edge_start_node',
       'edge_end_node', 'degree_x', 'degree_y', 'degree'],
      dtype='object')

In [52]:
result= result[['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION','gravity_to','severity','degree']]
result

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,gravity_to,severity,degree
0,07/12/2022,17:50,BROOKLYN,11225,40.663303,-73.960490,"(40.663303, -73.96049)",13633.263398,1.6,4
1,04/24/2022,1:30,BROOKLYN,11220,40.642986,-74.016210,"(40.642986, -74.01621)",10815.728466,1.6,6
2,04/24/2022,6:00,QUEENS,11411,0.000000,0.000000,"(0.0, 0.0)",0.000000,1.2,2
3,04/24/2022,21:40,BRONX,10452,40.843906,-73.924130,"(40.843906, -73.92413)",11655.760307,1.2,4
4,04/24/2022,17:45,BRONX,10466,40.894810,-73.861830,"(40.89481, -73.86183)",9294.127775,1.6,6
...,...,...,...,...,...,...,...,...,...,...
148204,02/06/2024,12:37,BROOKLYN,11235,40.586670,-73.966156,"(40.58667, -73.966156)",12581.834394,2.8,5
148205,03/05/2024,17:22,QUEENS,11436,40.680477,-73.792100,"(40.680477, -73.7921)",7140.039635,2.4,4
148206,03/05/2024,17:00,BROOKLYN,11204,40.610786,-73.978820,"(40.610786, -73.97882)",11224.031167,1.6,6
148207,03/05/2024,14:30,BROOKLYN,11207,40.677900,-73.892586,"(40.6779, -73.892586)",15976.493119,2.8,6


----

In [53]:
census = gpd.read_file("../01_data/03_intermediate/Cenpop/Cenpop.shp")

In [54]:
census

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,093,033102,36093033102,1400000US36093033102,331.02,Census Tract 331.02,G5020,S,183329990,3232878,+42.7780248,-074.1727005,"POLYGON ((-74.30655 42.75064, -74.30625 42.751..."
1,36,093,020102,36093020102,1400000US36093020102,201.02,Census Tract 201.02,G5020,S,854905,0,+42.8295375,-073.9161408,"POLYGON ((-73.92513 42.82029, -73.92440 42.821..."
2,36,093,032504,36093032504,1400000US36093032504,325.04,Census Tract 325.04,G5020,S,65999125,1264687,+42.9010539,-074.0288936,"POLYGON ((-74.09298 42.95587, -74.08805 42.955..."
3,36,081,003400,36081003400,1400000US36081003400,34,Census Tract 34,G5020,S,213987,0,+40.6847731,-073.8581871,"POLYGON ((-73.86629 40.68319, -73.86549 40.683..."
4,36,081,003900,36081003900,1400000US36081003900,39,Census Tract 39,G5020,S,146970,0,+40.7627994,-073.9409316,"POLYGON ((-73.94330 40.76066, -73.94247 40.761..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5406,36,027,130004,36027130004,1400000US36027130004,1300.04,Census Tract 1300.04,G5020,S,24394730,363637,+41.7909753,-073.8382016,"POLYGON ((-73.86601 41.81412, -73.86428 41.814..."
5407,36,027,050104,36027050104,1400000US36027050104,501.04,Census Tract 501.04,G5020,S,25421604,234551,+41.5929768,-073.7645636,"POLYGON ((-73.80389 41.58353, -73.80303 41.584..."
5408,36,119,014500,36119014500,1400000US36119014500,145,Census Tract 145,G5020,S,2716634,14648,+41.2850006,-073.9079663,"POLYGON ((-73.92411 41.27396, -73.92388 41.274..."
5409,36,061,016900,36061016900,1400000US36061016900,169,Census Tract 169,G5020,S,180002,0,+40.7851959,-073.9731196,"POLYGON ((-73.97644 40.78574, -73.97598 40.786..."


In [55]:
gdf = gpd.GeoDataFrame(result, geometry=gpd.points_from_xy(result['LONGITUDE'], result['LATITUDE']), crs='EPSG:4326')
census = census.to_crs(gdf.crs)

In [56]:
merged_gdf = gpd.sjoin(gdf, census, how="left", op="within")

/Users/lidouhao/anaconda3/envs/madina_env/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [57]:
merged_gdf.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'gravity_to', 'severity', 'degree', 'geometry',
       'index_right', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'GEOIDFQ',
       'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON'],
      dtype='object')

In [58]:
df = merged_gdf.dropna().drop(['index_right', 'STATEFP', 'COUNTYFP', 'TRACTCE','GEOIDFQ',
       'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT','INTPTLON','geometry'],axis = 1)

----

In [59]:
pop = pd.read_csv('../01_data/02_raw/nyc_decennialcensusdata_2010_2020_change-core-geographies.csv')
income = pd.read_csv('../01_data/02_raw/censustract-medianhouseholdincome2022.csv')

In [60]:
income['median_income'] = income['2014-2018'] 
income['GEOID'] = income['Census Tract'].astype(str)
income = income[['median_income','GEOID']]
result = pd.merge(df,income,on='GEOID', how = 'left')

In [61]:
pop['population'] = pop['Pop1'] 
pop['GEOID'] = pop['GeoID'].astype(str)
pop = pop[['population','GEOID']]
result = pd.merge(result,pop,on='GEOID', how = 'left')

In [62]:
result = result.dropna()
result.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,gravity_to,severity,degree,GEOID,median_income,population
0,07/12/2022,17:50,BROOKLYN,11225,40.663303,-73.960490,"(40.663303, -73.96049)",13633.263398,1.6,4,36047032700,45719.675776,"3,074"
1,04/24/2022,1:30,BROOKLYN,11220,40.642986,-74.016210,"(40.642986, -74.01621)",10815.728466,1.6,6,36047007400,39628.857024,"5,829"
2,04/24/2022,21:40,BRONX,10452,40.843906,-73.924130,"(40.843906, -73.92413)",11655.760307,1.2,4,36005020100,36481.442869,"4,345"
4,04/24/2022,0:00,BRONX,10453,40.861862,-73.912750,"(40.861862, -73.91275)",10339.244171,1.6,5,36005025700,48371.800056,"1,809"
5,03/08/2022,20:00,BROOKLYN,11207,40.666256,-73.900215,"(40.666256, -73.900215)",21481.276414,1.6,6,36047113400,27371.282629,"2,658"


In [63]:
result.shape

(127463, 13)

In [64]:
result.to_csv("../01_data/04_output/data.csv")